# YOLOv8 Bone Fracture Detection

## Downloading Bone Fracture Dataset

Your Kaggle API token can be downloaded from https://www.kaggle.com/settings/account

Retrieve the `Kaggle.json` file you downloaded and **make sure it is in the runtime directory** of the Colab notebook. I have automated this by storing the file inside my google drive and using ```drive.mount('/content/drive')```. If you are going to use this method, ensure you change the path to your file. Otherwise you can manually update the file and omitt the relevant code.

After this, we modify the values in the .yaml data file to reflect the new directory.

In [ ]:
!pip install -q opendatasets

from google.colab import drive
import os, sys
import opendatasets as od
import yaml

# Retrieving Kaggle API token from my google drive, change this if you plan on retrieving from elsewhere.
drive.mount('/content/drive')
kaggle_token_path = '/content/drive/MyDrive/AISI-Project/kaggle.json' # Replace with own path
!cp "$kaggle_token_path" . # Copy the kaggle.json file to the current working directory
drive.flush_and_unmount()

# Downloading dataset
od.download("https://www.kaggle.com/datasets/pkdarabi/bone-fracture-detection-computer-vision-project", data_dir='./datasets')


# Modifying YAML file to work with YOLOv8
yaml_path = '/content/datasets/bone-fracture-detection-computer-vision-project/data.yaml'

with open(yaml_path, 'r') as file:
    data = yaml.safe_load(file)

data['train'] = '/content/datasets/bone-fracture-detection-computer-vision-project/train/images'
data['val'] = '/content/datasets/bone-fracture-detection-computer-vision-project/valid/images'
data['test'] = '/content/datasets/bone-fracture-detection-computer-vision-project/test/images'

with open(yaml_path, 'w') as file:
    yaml.dump(data, file)

print("YAML file updated.")

Mounted at /content/drive


100%|██████████| 41.6M/41.6M [00:00<00:00, 199MB/s]



YAML file updated.


## Installing YOLOv8

In [ ]:
# Remove -q argument if things are going wrong, it just hides the pip output
!pip install -q git+https://github.com/ultralytics/ultralytics.git@main

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## Defining and Training YOLOv8 Model

In [ ]:
from ultralytics import YOLO

yaml_path = "./datasets/bone-fracture-detection-computer-vision-project/data.yaml"

# Custom Parameters
# Parameter | Value | Description:

# save	      True	  save train checkpoints and predict results
# batch	       -1    	number of images per batch (-1 for AutoBatch)
# patience	   50	    epochs to wait for no observable improvement for early stopping of training
# plots	      False	  save plots and images during train/val

model = YOLO('yolov8n.pt')
results = model.train(data=yaml_path, epochs=20, imgsz=600, batch=-1, plots=True, save=True)
#results = model.tune(data=yaml_path, epochs=30, imgsz=512, batch=-1, iterations=300, save=False)

Ultralytics YOLOv8.1.8 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./datasets/bone-fracture-detection-computer-vision-project/data.yaml, epochs=20, time=None, patience=50, batch=-1, imgsz=600, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False

train: Scanning /content/datasets/bone-fracture-detection-computer-vision-project/train/labels.cache... 3631 images, 1827 backgrounds, 0 corrupt: 100%|██████████| 3631/3631 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/datasets/bone-fracture-detection-computer-vision-project/valid/labels.cache... 348 images, 175 backgrounds, 0 corrupt: 100%|██████████| 348/348 [00:00<?, ?it/s]


Plotting labels to runs/detect/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.000640625), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 608 train, 608 val
Using 2 dataloader workers
Logging results to runs/detect/train2
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      9.59G      1.518      1.407      1.496         18        608: 100%|██████████| 45/45 [00:54<00:00,  1.21s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:03<00:00,  1.05s/it]

                   all        348        204      0.357      0.283       0.25     0.0907



20 epochs completed in 0.354 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.1.8 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3007013 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<00:00,  1.02it/s]


                   all        348        204      0.385      0.288      0.266     0.0947
        elbow positive        348         29      0.122      0.172     0.0784     0.0175
      fingers positive        348         48      0.302      0.167      0.119     0.0344
      forearm fracture        348         43      0.576      0.442      0.474       0.19
               humerus        348         36      0.772      0.556      0.611      0.222
     shoulder fracture        348         20      0.346       0.25      0.234      0.086
        wrist positive        348         28      0.193      0.143     0.0817     0.0177
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train2


## Model Prediction

In [ ]:
from ultralytics import YOLO

predict = model('/content/datasets/bone-fracture-detection-computer-vision-project/test/images/image2_199_png.rf.111ecddb2bdc3542d7f953385d1bb03b.jpg', visualize=True, save=True, save_txt=True)

## Exporting Model

In [ ]:
model.export(format='onnx')

## Downloading Results

In [ ]:
import shutil
from google.colab import files

TARGET_FOLDER = "/content/runs/detect/train"
OUTPUT_ZIP = "/content/output.zip"

# Create a zip file
shutil.make_archive(OUTPUT_ZIP.replace('.zip', ''), 'zip', TARGET_FOLDER)

# Download the file
files.download(OUTPUT_ZIP)
